In [5]:
%%configure -f
{"conf": {"spark.jars": "/jar/simpleHTM.jar"}}

In [6]:
import simple.HTMindex
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf

//var htmindex= new HTMindex()
//val htmid: (Double, Double) => Long = htmindex.lookupId(_,_)

//val htmidUDF=udf(htmid)

In [13]:
object HTMUtils extends Serializable {
    var htmindex= new HTMindex() with Serializable
    val htmid: (Double, Double) => Long = htmindex.lookupId(_,_)

    val htmidUDF=udf(htmid)
    
}

defined object HTMUtils


In [14]:
val dataset = Seq((123.0,45.0), (124.0,46.0)).toDF("ra", "dec")

In [21]:
dataset.withColumn("htmid", htmidUDF(dataset("ra"),dataset("dec"))).show

In [11]:
val datafile = "/user/hive/warehouse/source_test_parquet"
val old_df = spark.read.parquet(datafile)

old_df.createOrReplaceTempView("source")

val coordsDF = spark.sql("select coord_ra, coord_decl from source limit 10")
coordsDF.show()
coordsDF.printSchema()



datafile: String = /user/hive/warehouse/source_test_parquet
old_df: org.apache.spark.sql.DataFrame = [id: bigint, chunkid: int ... 86 more fields]
coordsDF: org.apache.spark.sql.DataFrame = [coord_ra: double, coord_decl: double]
+------------------+------------------+
|          coord_ra|        coord_decl|
+------------------+------------------+
| 339.8799009480773| 34.60159626500321|
| 339.9007831078834| 34.73830208759114|
| 339.8350724565858| 34.65489429025321|
| 339.9006981638171|34.741711297466075|
|339.85114223901024| 34.60391043664569|
| 339.9176012834339|34.733835899266445|
|339.85677629540703|34.596872654451175|
|340.03999850033125|  34.7732916046863|
|339.86516714065425|34.606816308674404|
| 339.9019848293452|34.727662874436874|
+------------------+------------------+

root
 |-- coord_ra: double (nullable = true)
 |-- coord_decl: double (nullable = true)



In [14]:
val new_df = coordsDF.withColumn("htmid",HTMUtils.htmidUDF(old_df("coord_ra"),old_df("coord_decl")))
new_df.show(1)

new_df: org.apache.spark.sql.DataFrame = [coord_ra: double, coord_decl: double ... 1 more field]
+------------------+-----------------+--------------+
|          coord_ra|       coord_decl|         htmid|
+------------------+-----------------+--------------+
|357.03448646379684|33.41077013684323|13319061131005|
+------------------+-----------------+--------------+
only showing top 1 row

